In [1]:
!pip install openai -q

In [3]:
import os

from dotenv import load_dotenv
from openai import OpenAI
import json

load_dotenv()

# Load the API key from the .env file
OPENAI_KEY = os.getenv("OPENAI_KEY")

 
client = OpenAI(api_key=OPENAI_KEY)

## Get the current weather with Static Data

In [4]:
def get_current_weather(location, unit="Fahrenheit"):
    """
    Get the current weather of a location
    :param location: str: The location of the weather
    :param unit: str: The unit of the temperature
    :return: str: The current weather
    """

    weather_info = {
        "location": location,
        "unit": unit,
        "temperature": "70",
        "forcast": ["sunny", "clear sky", "windy"]
    }
    return json.dumps(weather_info)

## Append messages to the chatbot

In [5]:
messages = [
    {"role": "user", "content": "what is the live weather in boston?"}
]

## Functions

In [6]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather of a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state e.g. Boston, MA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["Fahrenheit", "Celsius"],
                },
            },
            "required": ["location"],
        }
    }
]

## Call the completion endpoint

In [9]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we explicitly set it here
)

response_messages = response.choices[0].message
print(response_messages)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None)


## Extract the function name and arguments

In [10]:
function_name = response.choices[0].message.function_call.name
function_args = json.loads(response.choices[0].message.function_call.arguments)

## Get the function to call

In [13]:
available_functions = {
    "get_current_weather": get_current_weather
}

function_to_call = available_functions[function_name]

## Call the function

In [16]:
function_response = function_to_call(location=function_args.get("location"))
print(function_response)

{"location": "Boston, MA", "unit": "Fahrenheit", "temperature": "70", "forcast": ["sunny", "clear sky", "windy"]}


In [20]:
messages.append(response_messages)
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": function_response
    }
)
print(messages)

[{'role': 'user', 'content': 'what is the live weather in boston?'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None), {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "Boston, MA", "unit": "Fahrenheit", "temperature": "70", "forcast": ["sunny", "clear sky", "windy"]}'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None), {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "Boston, MA", "unit": "Fahrenheit", "temperature": "70", "forcast": ["sunny", "clear sky", "windy"]}'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None), {'role': 'function', 'name

In [19]:
final_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)

print(final_response.choices[0].message.content)

The current weather in Boston, MA is 70°F with sunny and clear skies. It is also windy.
